In [4]:
%reload_ext autoreload
%autoreload 2
import logging
import grpc
import pandas as pd
import sys
sys.path.append('proto')
from proto import contest_pb2
from proto import contest_pb2_grpc
from proto import question_pb2
from proto import question_pb2_grpc
import numpy as np
import time
from utils import *
from strategy_utils import *

In [5]:
contest_channel=grpc.insecure_channel('47.103.23.116: 56702')
question_channel=grpc.insecure_channel('47.103.23.116: 56701')

contest_stub=contest_pb2_grpc.ContestStub(contest_channel)
question_stub=question_pb2_grpc.QuestionStub(question_channel)

login_response=contest_stub.login(contest_pb2.LoginRequest(user_id=88,user_pin='dDTSvdwk'))

session_key=login_response.session_key
init_capital=login_response.init_capital

In [14]:
i=0#控制seq
count=0
data_lst=[]
factors_lst=[]
period=1#eg 每两天跑一次策略
comission=0#
max_exposure=0.1#大盘上涨，多头增加，大盘下跌，空头增加
single_stock_position_limit=0.1#
lending_rate=0.01#
borrow_rate=0.05#
leverage=2#
factors=pd.DataFrame()
prev_factors=pd.DataFrame()#上一次计算的factors

In [17]:
while True:
    question_response=question_stub.get_question(question_pb2.QuestionRequest(user_id=88,sequence=i))
    print(question_response.sequence)
    if question_response.sequence!=-1:
        dailystk = [x.values for x in question_response.dailystk]
        data_lst.extend(dailystk)

        if count>10:#开始不动，只要有新数据就跑一次策略
            print('run strategy')
            df=pd.DataFrame(data_lst,columns=['day','stock','open','high','low','close','volume'],
                            dtype=float).set_index(['day','stock'])
            
            dailyfactors=get_factors(df,prev_factors)  #从数据获取因子
            prev_factors=dailyfactors

            factors_lst.extend(dailyfactors.values)#向因子库追加
            #部份因子可能需要根据历史因子数据归一化

            # factor_select=select_factors(factors,n=10,period=period)  #计算相关系数选取因子
            # factor_select=['avg', 'mom', 'max52', 'cci', 'K', 'D', 'J', 'rsi', 'trix', 'willr']
            # factor_select=['avg','mom', 'vol', 'max52', 'K', 'D', 'J', 'rsi', 'trix', 'willr', 'macd', 'natr','mfi']
            factor_select=['avg', 'mom', 'max52', 'D', 'willr', 'natr', 'rsi']

            # index_direction='neutral'#TODO 大盘方向，用于控制exposure

            weights = get_weight(dailyfactors[factor_select],head_n=10,tail_n=10)
            if count < 100:
                target_pos=get_position(weights,
                                        pd.DataFrame(dailystk,columns=['day','stock','open','high','low','close','volume'],dtype=float),#只需要close，待优化
                                        question_response.positions,
                                        question_response.capital / 2,
                                        comission)
            else:
                target_pos=get_position(weights,
                                        pd.DataFrame(dailystk,columns=['day','stock','open','high','low','close','volume'],dtype=float),#只需要close，待优化
                                        question_response.positions,
                                        question_response.capital,
                                        comission)
            
            #提交策略
            if count%period==0:#按周期提交
                ##summit answer
                submit_response = contest_stub.submit_answer(contest_pb2.AnswerRequest(user_id=88,user_pin='dDTSvdwk',session_key=login_response.session_key,sequence=i,positions=target_pos))

                print(submit_response,question_response.capital)
                if not submit_response.accepted:
                    print(submit_response.reason)
                    if submit_response.reason[-7:] == 'timeout':
                        i=question_response.sequence+1
                        count+=1
                        continue#如果提交超时，直接请求新数据

        i=question_response.sequence+1
        count+=1

        # if count==20:
            # break

    time.sleep(1)

1002.0     346311
1003.0    -299977
1006.0     -90447
1007.0      28395
           ...   
1317.0     387928
1327.0    1634435
1334.0      -4966
1346.0     391343
1349.0      38753
Name: close, Length: 103, dtype: int32
reason: "session key not match"
 484132917.65911025
session key not match
-1
-1
-1
588
run strategy
stock
1001.0     196211
1002.0     172987
1003.0    -303315
1006.0     -91523
1008.0     521226
           ...   
1327.0    1651061
1333.0      21937
1334.0      -4962
1346.0     391738
1349.0      39051
Name: close, Length: 107, dtype: int32
reason: "session key not match"
 481647640.31157947
session key not match
-1
-1
-1
-1
589
run strategy
stock
1001.0     197862
1002.0     179063
1003.0    -158423
1005.0     -61209
1006.0     -93342
           ...   
1317.0     396385
1327.0    1690466
1334.0      -5006
1346.0     403481
1349.0      80843
Name: close, Length: 101, dtype: int32
reason: "session key not match"
 477969332.88442725
session key not match
-1
-1
-1
590
run s

KeyboardInterrupt: 

In [9]:
IF_REV = {
    'avg':True,
    'mom':True,
    'vol':False,
    'max52':True,
    'min52':True,
    'cci':True,
    'K':True,
    'D':True,
    'J':False,
    'rsi':True,
    'trix':True,
    'willr':False,
    'macd':True,
    'natr':False,
    'mfi':True
    }
def get_weight(dailyfactors,head_n=10,tail_n=10):
    
    all_weight = np.empty((0,dailyfactors.shape[0]))
    for fac in dailyfactors:
        head = dailyfactors[fac].nlargest(head_n).index.tolist()
        tail = dailyfactors[fac].nsmallest(tail_n).index.tolist()
        weight = np.zeros(351)
        weight[head] = 1 / head_n
        weight[tail] = -1 / tail_n
        if IF_REV[fac]:
            weight = weight * -1
        all_weight = np.vstack([all_weight,weight])
    sum_weight = all_weight.mean(axis=0)
    sum_weight[sum_weight>0] = sum_weight[sum_weight>0] / sum_weight[sum_weight>0].sum()
    sum_weight[sum_weight<0] = sum_weight[sum_weight<0] / sum_weight[sum_weight<0].sum() * -1
    return sum_weight

In [27]:
sum_weight[sum_weight>0].sum()

0.6375

In [55]:
sum_weight = all_weight.mean(axis=0)

In [57]:
sum_weight[sum_weight>0] = sum_weight[sum_weight>0] / sum_weight[sum_weight>0].sum()
sum_weight[sum_weight<0] = sum_weight[sum_weight<0] / sum_weight[sum_weight<0].sum() * -1

In [58]:
sum_weight[sum_weight>0].sum()

0.9999999999999999

In [59]:
sum_weight[sum_weight<0].sum()

-0.9999999999999998